In [ ]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = "https://datasets.imdbws.com/title.basics.tsv.gz"
df3 = pd.read_csv(url, sep = '\t', low_memory=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11264054 entries, 0 to 11264053
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 773.4+ MB
None


In [ ]:
display(df3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
11270714,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2009,\N,\N,"Action,Drama,Family"
11270715,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
11270716,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
11270717,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [ ]:
print(df3.dtypes)

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object


In [ ]:
# Résumé statistique des colonnes numériques
print(df3.describe())


           tconst  titleType  primaryTitle originalTitle   isAdult startYear  \
count    11264054   11264054      11264035      11264035  11264054  11264054   
unique   11264054         11       5056878       5081559        42       152   
top     tt0000001  tvEpisode  Episode #1.1  Episode #1.1         0        \N   
freq            1    8652927         53496         53496  10901448   1417127   

         endYear runtimeMinutes    genres  
count   11264054       11264054  11263421  
unique        98            945      2383  
top           \N             \N     Drama  
freq    11131490        7707910   1276745  


In [ ]:
# Vérifier les valeurs manquantes dans chaque colonne
print(df3.isnull().sum())

NameError: name 'df3' is not defined

In [ ]:
# Filtrer les films uniquement (titleType == 'movie')
df_films = df3[df3['titleType'] == 'movie']

In [ ]:
# Vérifier le nombre de NaN avant le remplacement
nan_before_runtime = df_films['runtimeMinutes'].isna().sum()
nan_before_startYear = df_films['startYear'].isna().sum()
nan_before_endYear = df_films['endYear'].isna().sum()

print(f"NaN avant remplacement (runtimeMinutes) : {nan_before_runtime}")
print(f"NaN avant remplacement (startYear) : {nan_before_startYear}")
print(f"NaN avant remplacement (endYear) : {nan_before_endYear}")

# Remplacer les valeurs '\N' par NaN
df_films.loc[:, 'runtimeMinutes'] = df_films['runtimeMinutes'].replace(r'\\N', np.nan, regex=True)
df_films.loc[:, 'startYear'] = df_films['startYear'].replace(r'\\N', np.nan, regex=True)
df_films.loc[:, 'endYear'] = df_films['endYear'].replace(r'\\N', np.nan, regex=True)

# Vérifier le nombre de NaN après le remplacement
nan_after_runtime = df_films['runtimeMinutes'].isna().sum()
nan_after_startYear = df_films['startYear'].isna().sum()
nan_after_endYear = df_films['endYear'].isna().sum()

print(f"NaN après remplacement (runtimeMinutes) : {nan_after_runtime}")
print(f"NaN après remplacement (startYear) : {nan_after_startYear}")
print(f"NaN après remplacement (endYear) : {nan_after_endYear}")


NaN avant remplacement (runtimeMinutes) : 258602
NaN avant remplacement (startYear) : 101513
NaN avant remplacement (endYear) : 698281
NaN après remplacement (runtimeMinutes) : 258602
NaN après remplacement (startYear) : 101513
NaN après remplacement (endYear) : 698281


In [ ]:
# Convertir les colonnes startYear, endYear et runtimeMinutes en numériques
df_films['startYear'] = pd.to_numeric(df_films['startYear'], errors='coerce')
df_films['endYear'] = pd.to_numeric(df_films['endYear'], errors='coerce')
df_films['runtimeMinutes'] = pd.to_numeric(df_films['runtimeMinutes'], errors='coerce')

<ipython-input-13-5da5b961dcdb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_films['startYear'] = pd.to_numeric(df_films['startYear'], errors='coerce')
<ipython-input-13-5da5b961dcdb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_films['endYear'] = pd.to_numeric(df_films['endYear'], errors='coerce')
<ipython-input-13-5da5b961dcdb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
# Convertir les colonnes en numériques avec .loc[] pour éviter le warning
df_films.loc[:, 'startYear'] = pd.to_numeric(df_films['startYear'], errors='coerce')
df_films.loc[:, 'endYear'] = pd.to_numeric(df_films['endYear'], errors='coerce')
df_films.loc[:, 'runtimeMinutes'] = pd.to_numeric(df_films['runtimeMinutes'], errors='coerce')


In [ ]:
# Convertir 'isAdult' en booléen (0 = False, 1 = True)
df_films['isAdult'] = df_films['isAdult'].astype(bool)

<ipython-input-15-d9c65bc0b866>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_films['isAdult'] = df_films['isAdult'].astype(bool)


In [ ]:
# Convertir 'isAdult' en booléen (0 = False, 1 = True)
df_films.loc[:, 'isAdult'] = df_films['isAdult'].astype(int) == 1


In [ ]:
# Supprimer les films sans durée valide (runtimeMinutes NaN)
df_films_cleaned = df_films.dropna(subset=['runtimeMinutes'])

In [ ]:
# Afficher les premières lignes du dataset nettoyé
print(df_films_cleaned.head().to_markdown())

|     | tconst    | titleType   | primaryTitle                  | originalTitle                 | isAdult   |   startYear | endYear   |   runtimeMinutes | genres                     |
|----:|:----------|:------------|:------------------------------|:------------------------------|:----------|------------:|:----------|-----------------:|:---------------------------|
|   8 | tt0000009 | movie       | Miss Jerry                    | Miss Jerry                    | False     |        1894 | \N        |               45 | Romance                    |
| 144 | tt0000147 | movie       | The Corbett-Fitzsimmons Fight | The Corbett-Fitzsimmons Fight | False     |        1897 | \N        |              100 | Documentary,News,Sport     |
| 498 | tt0000502 | movie       | Bohemios                      | Bohemios                      | False     |        1905 | \N        |              100 | \N                         |
| 570 | tt0000574 | movie       | The Story of the Kelly Gang   | The Story of t

In [ ]:
# Afficher le résumé des types de données après nettoyage
print(df_films_cleaned.dtypes)

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object


In [ ]:
# Vérifier les valeurs manquantes restantes
print(df_films_cleaned.isnull().sum())

tconst            0
titleType         0
primaryTitle      2
originalTitle     2
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64


In [ ]:
df_films_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698552 entries, 8 to 11270669
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          698552 non-null  object
 1   titleType       698552 non-null  object
 2   primaryTitle    698550 non-null  object
 3   originalTitle   698550 non-null  object
 4   isAdult         698552 non-null  object
 5   startYear       698552 non-null  object
 6   endYear         698552 non-null  object
 7   runtimeMinutes  698552 non-null  object
 8   genres          698552 non-null  object
dtypes: object(9)
memory usage: 53.3+ MB


In [ ]:
df_films_cleaned = df_films_cleaned.drop('endYear', axis=1)



In [ ]:
df_films_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698552 entries, 8 to 11270669
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          698552 non-null  object
 1   titleType       698552 non-null  object
 2   primaryTitle    698550 non-null  object
 3   originalTitle   698550 non-null  object
 4   isAdult         698552 non-null  object
 5   startYear       698552 non-null  object
 6   endYear         698552 non-null  object
 7   runtimeMinutes  698552 non-null  object
 8   genres          698552 non-null  object
dtypes: object(9)
memory usage: 53.3+ MB


In [ ]:
d3 = df_films_cleaned
print(d3.info())


<class 'pandas.core.frame.DataFrame'>
Index: 698552 entries, 8 to 11270669
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          698552 non-null  object
 1   titleType       698552 non-null  object
 2   primaryTitle    698550 non-null  object
 3   originalTitle   698550 non-null  object
 4   isAdult         698552 non-null  object
 5   startYear       698552 non-null  object
 6   endYear         698552 non-null  object
 7   runtimeMinutes  698552 non-null  object
 8   genres          698552 non-null  object
dtypes: object(9)
memory usage: 53.3+ MB
None
